In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
is_data_loaded = True
#is_data_loaded = False



In [55]:
fold = "5"

train_src_name = 'train_fold_'+fold
test_src_name = 'test_fold_'+fold
print("Using local data")
train_data = pd.read_csv('./'+train_src_name+'.csv')



test_data = pd.read_csv('./'+test_src_name+'.csv')


alt_train_data = pd.read_csv('./'+train_src_name+'.csv')

Using local data


In [56]:
len(train_data)

1383

In [57]:
len(test_data)

345

## Preprocess Train Data and Finally split to valid

> Add blockquote



In [58]:
# Drop missing values
train_data.dropna(inplace=True)





In [59]:
label_col = 'label'
cf_label_val = 'negative'
org_label_val = 'positive'


print("Label Column: "+label_col)
# Separate features and labels
X_train_data = train_data.drop(label_col, axis=1)
y_train_data = train_data[label_col].apply(lambda x: 1 if x.strip() == cf_label_val else 0)  # Convert labels to binary

# Define preprocessing for numerical and categorical columns
numerical_features = X_train_data.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train_data.select_dtypes(include=['object']).columns

Label Column: label


In [60]:
# Preprocessor (fit only on training data)
preprocessor = ColumnTransformer(transformers=[('num', StandardScaler(), numerical_features),
          ('cat', OneHotEncoder(drop='first',handle_unknown='ignore'), categorical_features)])

In [61]:
# Apply transformations
X_train_data = preprocessor.fit_transform(X_train_data)

# Convert sparse matrix to dense if necessary
if hasattr(X_train_data, 'toarray'):  # Check if X is sparse
    X_train_data = X_train_data.toarray()


print("Train set transformed successfully!")

X_train = X_train_data
y_train = y_train_data

Train set transformed successfully!


In [62]:
len(y_train_data)

1383

## Preprocess Test Data

In [63]:
# Now apply the same transformations to the test set
test_data.dropna(inplace=True)

# Separate features and labels for the test set
X_test_data = test_data.drop(label_col, axis=1)
y_test_data = test_data[label_col].apply(lambda x: 1 if x.strip() == cf_label_val else 0)  # Convert labels to binary

# Transform the test data using the already fitted preprocessor
X_test_data = preprocessor.transform(X_test_data)

# Convert sparse matrix to dense if necessary for the test set
if hasattr(X_test_data, 'toarray'):
    X_test_data = X_test_data.toarray()

# Now you can use X_test and y_test in your model evaluation or testing
print("Test set transformed successfully!")

X_test = X_test_data
y_test = y_test_data

Test set transformed successfully!


## Gradient Boosted Classifier

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Initialize the classifier
# Initialize the Logistic Regression model
lr = LogisticRegression()

# Train the model on the training data
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.9565217391304348
              precision    recall  f1-score   support

           0       0.92      0.94      0.93       103
           1       0.97      0.96      0.97       242

    accuracy                           0.96       345
   macro avg       0.94      0.95      0.95       345
weighted avg       0.96      0.96      0.96       345

Confusion Matrix:
[[ 97   6]
 [  9 233]]


## Saving data to file

In [65]:
def evaluate_and_save_lr(model, X_train, original_train_data, output_file='train_predictions_gbc.csv'):

  # Get the model's predictions (using probability threshold of 0.5 for binary classification)
  y_pred = model.predict(X_train)


  # Convert binary predictions to the original format ('>50K' and '<=50K')
  y_pred_str = [cf_label_val if y == 1 else org_label_val for y in y_pred]

  # Add predictions to the original DataFrame
  original_train_data['predictions'] = y_pred_str

  # Save the combined DataFrame with predictions to a CSV file
  original_train_data.to_csv(output_file, index=False)

  print(f"Predictions saved to {output_file}")

In [66]:
# Example usage:
# original_test_data is the DataFrame containing the original test data (with features, but without labels)
# test_loader is the DataLoader for the test set
evaluate_and_save_lr(lr, X_train, train_data, output_file='lr_pred_'+train_src_name+'.csv')

Predictions saved to lr_pred_train_fold_5.csv


In [67]:
evaluate_and_save_lr(lr, X_test, test_data, output_file='pred_'+test_src_name+'.csv')

Predictions saved to pred_test_fold_5.csv
